In [ ]:
import tkinter as tk
from tkinter import filedialog, scrolledtext, messagebox
from PIL import Image, ImageTk
from openai import OpenAI
import base64
import os

# ======================
# OpenRouter Client Setup
# ======================
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="",  # 🔑 Replace with your OpenRouter API key
)

# ======================
# Encode Image to Base64
# ======================
def encode_image(image_path):
    try:
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to encode image: {e}")
        return None

# ======================
# Generate Caption
# ======================
def generate_caption(image_path):
    image_b64 = encode_image(image_path)
    if not image_b64:
        return None

    try:
        completion = client.chat.completions.create(
            model="openai/gpt-4o-mini",  # Vision model
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Generate a short caption for this image:"},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"}}
                    ]
                }
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        messagebox.showerror("Error", f"Caption generation failed: {e}")
        return None

# ======================
# Generate Stories (EN, HI, MR)
# ======================
def generate_stories(caption):
    try:
        completion = client.chat.completions.create(
            model="deepseek/deepseek-r1:free",
            messages=[
                {
                    "role": "user",
                    "content": f"""The image caption is: "{caption}".

Now, write 3 short stories based on it:
1. In **English**,
2. In **Hindi**,
3. In **Marathi**.

Format clearly with headings for each language.
"""
                }
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        messagebox.showerror("Error", f"Story generation failed: {e}")
        return None

# ======================
# GUI Functions
# ======================
def upload_image():
    global selected_image, img_label, img_display
    file_path = filedialog.askopenfilename(
        title="Select Image",
        filetypes=[("Image Files", "*.jpg *.jpeg *.png *.webp")]
    )
    if file_path:
        selected_image = file_path
        status_label.config(text=f"Selected: {os.path.basename(file_path)} ✅")

        # Show image in GUI
        img = Image.open(file_path)
        img.thumbnail((300, 300))  # Resize for display
        img_display = ImageTk.PhotoImage(img)
        img_label.config(image=img_display)
        img_label.image = img_display

def process_image():
    if not selected_image:
        messagebox.showwarning("Warning", "Please upload an image first!")
        return

    status_label.config(text="⏳ Generating caption...")
    root.update_idletasks()

    caption = generate_caption(selected_image)
    if caption:
        output_text.delete(1.0, tk.END)  # Clear previous text
        output_text.insert(tk.END, f"📌 Caption:\n{caption}\n\n")
        status_label.config(text="✅ Caption generated. Now creating stories...")

        stories = generate_stories(caption)
        if stories:
            output_text.insert(tk.END, f"{stories}\n\n")
            status_label.config(text="✅ Done!")

# ======================
# Tkinter GUI
# ======================
root = tk.Tk()
root.title("🖼️ Image Caption & Story Generator")
root.geometry("800x700")

selected_image = None
img_display = None

# Upload & Process Buttons
upload_btn = tk.Button(root, text="📂 Upload Image", command=upload_image, width=20, bg="#4CAF50", fg="white")
upload_btn.pack(pady=10)

process_btn = tk.Button(root, text="🚀 Generate Caption & Stories", command=process_image, width=30, bg="#2196F3", fg="white")
process_btn.pack(pady=10)

# Status Label
status_label = tk.Label(root, text="No image selected", fg="gray")
status_label.pack(pady=5)

# Image Display
img_label = tk.Label(root)
img_label.pack(pady=10)

# Output Box
output_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=90, height=20, font=("Arial", 11))
output_text.pack(pady=10)

root.mainloop()


KeyboardInterrupt: 